In [1]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import pandas as pd
import numpy as np
from functions import *

client = oandapyV20.API(access_token='199d815eb1f94c176a86dd3d5cc991a5-5e6ff7a15c524bf65fc0af5ae9a11972')

instrument = "EUR_USD"
params = {
    "from": "2020-12-16T00:00:00Z",
    "to": "2021-01-15T00:00:00Z",
    "granularity": "M1",
    "count": 5000,
}

fields = ['year', 'month', 'day', 'hour', 'minute', 'second', 'open', 'high', 'low', 'close', 'volume']

In [104]:
get_data(instrument, params, 'minute_data.csv')

In [96]:
params['granularity'] = 'M5'
get_data(instrument, params, 'five_minute_data.csv')

In [65]:
min_data = pd.read_csv('minute_data.csv')

In [66]:
samples = len(min_data)
data = min_data[['open', 'high', 'low', 'close', 'volume']].to_numpy()
data = data.astype(np.float64)
training_size = int(samples*0.7)
test_size = samples-training_size
lookback = 600
trainY=[]
for i in range(training_size-lookback-1440):
    close = data[i+lookback-1][3]
    up = True
    down = True
    for j in range(1440):
        if down and data[i+lookback+j][1]-close>=0.0010:
            down = False
        elif up and data[i+lookback+j][2]-close<=-0.0010:
            up = False
        if data[i+lookback+j][1]-close>=0.0030 and up:
            trainY.append(np.array([0, 0, 1]))
            break
        if data[i+lookback+j][2]-close<=-0.0030 and down:
            trainY.append(np.array([1, 0, 0]))
            break
        if not down and not up:
            trainY.append(np.array([0, 1, 0]))
            break
    if len(trainY)!=i+1:
        trainY.append(np.array([0, 1, 0]))
trainY = np.array(trainY)

In [67]:
testY=[]
for i in range(training_size, samples-lookback-1440):
    close = data[i+lookback-1][3]
    up = True
    down = True
    for j in range(1440):
        if down and data[i+lookback+j][1]-close>=0.0010:
            down = False
        elif up and data[i+lookback+j][2]-close<=-0.0010:
            up = False
        if data[i+lookback+j][1]-close>=0.0030 and up:
            testY.append(np.array([0, 0, 1]))
            break
        if data[i+lookback+j][2]-close<=-0.0030 and down:
            testY.append(np.array([1, 0, 0]))
            break
        if not down and not up:
            testY.append(np.array([0, 1, 0]))
            break
    if len(testY)!=i+1-training_size:
        testY.append(np.array([0, 1, 0]))
testY = np.array(testY)

In [68]:
np.sum(trainY, axis=0)

array([3120, 8961, 5604])

In [17]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
data1 = data.copy()
scale = MinMaxScaler().fit(data1[:training_size, :3])
data1[:, :3] = scale.transform(data1[:, :3])

In [69]:
trainX, testX = [], []
for i in range(training_size-lookback-1440):
    a = data[i:i+lookback]
    trainX.append(a)
trainX = np.array(trainX)

for i in range(training_size, samples-lookback-1440):
    a = data[i:i+lookback]
    testX.append(a)
testX = np.array(testX)

In [70]:
import tensorflow as tf
import keras as k
checkpoint_path = 'training/_1/no_scaling.ckpt'
cp_callback = k.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [71]:
model = make_model(lookback)
model.fit(trainX, trainY, batch_size=100, epochs=10, validation_data=(testX, testY), callbacks=[cp_callback])

Epoch 1/10
177/177 [==============================] - 509s 3s/step - loss: 1.0501 - acc: 0.4842 - val_loss: 1.1118 - val_acc: 0.4897

In [102]:
model = make_model(lookback)
model.load_weights('training/_3/feature_scaling_min_max.ckpt')

In [112]:
p2 = model.predict(trainX)

In [117]:
ans = []
total_times_invested=0
for i in p2:
    if 0.5 < i[0] < 0.6:
        ans.append([1, 0, 0])
        total_times_invested+=1
    elif 0.6>i[2]>0.5:
        ans.append([0, 0, 1])
        total_times_invested+=1
    else:
        ans.append([0, 1, 0])
ans = np.array(ans)

In [118]:
correctly_invested=0
for i in range(len(ans)):
    if np.array_equal(ans[i], trainY[i]) and (np.array_equal(ans[i], [1, 0, 0]) or np.array_equal(ans[i], [0, 0, 1])):
        correctly_invested+=1
print(correctly_invested)
print(total_times_invested)
print(float(correctly_invested)/total_times_invested)

1083
2928
0.3698770491803279
